In [ ]:
# Install the dependencies
!pip install tensorflow==2.12.0
!pip install tensorflow-hub
!pip install pillow
!pip install -U -q tensorflow-datasets

In [2]:
# Step 1: Import necessary libraries
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
# Step 2: Load the Food-101 Dataset
# Store the unprocessed raw data
(train_data_raw, test_data_raw), info = tfds.load(
    'food101',
    split=['train', 'validation'],
    with_info=True,
    as_supervised=True,
)
num_classes = info.features['label'].num_classes

In [ ]:
# Step 3: Data preprocessing and augmentation
image_size = (128, 128)
BATCH_SIZE = 8
BUFFER_SIZE = 1000

# Calculate the total number of training samples
total_train_samples = len(train_data_raw)
steps_per_epoch = total_train_samples // BATCH_SIZE

def preprocess(image, label):
  """Preprocesses an image and label for the model."""
  image = tf.image.convert_image_dtype(image, dtype=tf.float32)  # Converts image data type to float32
  # Check if image has 3 channels (RGB)
  if image.shape[-1] != 3:
      image = tf.expand_dims(image, axis=-1)  # Add a channel dimension if it's missing (for grayscale images)
      image = tf.image.grayscale_to_rgb(image)  # Convert grayscale to RGB using tf.image.grayscale_to_rgb
  image = tf.image.resize(image, image_size)  # Convert image to consistent size
  label = tf.one_hot(label, num_classes)  # Converts class labels to vectors, to use categorical_crossentropy loss
  return image, label

data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),  # Rotate the image horizontally
        layers.RandomRotation(0.1),       # Randomly rotates images by a small angle, done to improve generalization
    ]
)

# Define the data generator function
def data_generator(dataset):
    """Yields preprocessed images and labels from the dataset. Yield ensures data loaded on demand"""
    for image, label in dataset:
        yield preprocess(image, label)

# Create the tf.data.Dataset for training data using the generator
train_data = tf.data.Dataset.from_generator(
    lambda: data_generator(train_data_raw),  # Using train_data_raw here
    output_types=(tf.float32, tf.float32),
    output_shapes=((128, 128, 3), (num_classes,))
).batch(BATCH_SIZE).prefetch(BUFFER_SIZE)

# These lines preprocess the data for the test/validation set
# (consider using a generator here as well if you're still facing memory constraints)
test_data = test_data_raw.map(preprocess).batch(BATCH_SIZE).prefetch(BUFFER_SIZE)

# Enable mixed precision training
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

In [5]:
# Step 4: # Build the model (using transfer learning with EfficientNetB0)
def build_model():
  base_model = tf.keras.applications.EfficientNetB0(
      include_top=False, weights='imagenet', input_shape=(128, 128, 3)
  )

  base_model.trainable = False # Freeze weights of base model initially to prevent large changes during training

  model = keras.Sequential([
      base_model,
      data_augmentation,
      layers.GlobalAveragePooling2D(),
      layers.Dropout(0.2),  # Randomly to drop out neurons during training to prevent overfitting
      layers.Dense(num_classes, activation='softmax')
  ])
  return model

model = build_model()

In [ ]:
# Step 5: Compile and Train the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Train the model with garbage collection
epochs = 10
for epoch in range(epochs):
    history = model.fit(train_data, epochs=1, steps_per_epoch=steps_per_epoch, validation_data=test_data)
    gc.collect()  # Force garbage collection after each epoch

In [ ]:
# Step 6: Evaluate the model
loss, accuracy = model.evaluate(test_data)
print('Test accuracy:', accuracy)

In [ ]:
# Visualize Training History
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.show()